In [ ]:
pip install --upgrade keras-cv

In [ ]:
from google.colab import drive
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np

drive.mount('/content/drive')


read_from_drive = True

if read_from_drive:
  drive.mount('/content/drive/')
  train_path = '/content/drive/MyDrive/COMP90086_2024_Project_train'
  test_path = '/content/drive/MyDrive/COMP90086_2024_Project_test'
else:
  train_path = "./COMP90086_2024_Project_train"
  test_path = "./COMP90086_2024_Project_test"



Mounted at /content/drive
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# the parameters
DIM = 224
EPOCHS = 40
BATCH = 32
DROPOUT = 0.3
LEARNING = 2e-5

In [ ]:
train_df = pd.read_csv(os.path.join(train_path, "train.csv"))
test_df = pd.read_csv(os.path.join(test_path, "test.csv"))

train_df['id'] = train_df['id'].astype(str)
train_df.sort_values("id", inplace=True)

train_labels = list(train_df['stable_height'])


image_dataset = tf.keras.utils.image_dataset_from_directory(
    train_path,
    labels=train_labels,
    label_mode='categorical',
    validation_split=0.2,
    subset='both',
    seed=42,
    image_size=(DIM, DIM),
    shuffle = False,
    batch_size= BATCH
)


train_dataset = image_dataset[0]
validation_dataset = image_dataset[1]

Found 7680 files belonging to 6 classes.
Using 6144 files for training.
Using 1536 files for validation.


In [ ]:
num_distinct_labels = len(set(train_labels))
print("Number of distinct labels:", num_distinct_labels)


Number of distinct labels: 6


In [ ]:
from tensorflow.keras.applications import VGG19

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Make sure the weights are not freezed
for layer in base_model.layers:
  layer.trainable = True


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling, Input, RandomContrast, RandomBrightness, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential([
    Rescaling(1./255),
    RandomContrast(0.2),
    RandomBrightness(0.2),
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(DROPOUT),
    Dense(64, activation='relu'),
    Dropout(DROPOUT),
    Dense(6, activation='softmax')
])



model.compile(optimizer= keras.optimizers.Adam(learning_rate=LEARNING),
              loss= 'categorical_crossentropy',
              metrics=['accuracy'])


checkpoint = ModelCheckpoint(
    filepath='/content/drive/MyDrive/best_model.weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    verbose=1
)


history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=validation_dataset,
    callbacks=[checkpoint]
)


Epoch 1/40
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 933ms/step - accuracy: 0.2174 - loss: 1.6679
Epoch 1: val_accuracy improved from -inf to 0.25065, saving model to /content/drive/MyDrive/best_model.weights.h5
192/192 ━━━━━━━━━━━━━━━━━━━━ 203s 969ms/step - accuracy: 0.2174 - loss: 1.6677 - val_accuracy: 0.2507 - val_loss: 1.5325
Epoch 2/40
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.2312 - loss: 1.5776
Epoch 2: val_accuracy did not improve from 0.25065
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 75ms/step - accuracy: 0.2312 - loss: 1.5775 - val_accuracy: 0.2370 - val_loss: 1.5272
Epoch 3/40
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.2346 - loss: 1.5735
Epoch 3: val_accuracy did not improve from 0.25065
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 74ms/step - accuracy: 0.2347 - loss: 1.5734 - val_accuracy: 0.2409 - val_loss: 1.5236
Epoch 4/40
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.2274 - loss: 1.5674
Epoch 4: val_accuracy did not improve from 0.25065
192/192 ━━━━━━━━━━━━━━━━━

In [ ]:
# Load the trained model weight
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling, Input, RandomContrast, RandomBrightness, GlobalAveragePooling2D
# from tensorflow.keras.callbacks import ModelCheckpoint

# model = Sequential([
#     Rescaling(1./255),
#     RandomContrast(0.2),
#     RandomBrightness(0.2),
#     base_model,
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(DROPOUT),
#     Dense(64, activation='relu'),
#     Dropout(DROPOUT),
#     Dense(6, activation='softmax')
# ])

# model.load_weights('/content/drive/MyDrive/best_model.weights.h5')

In [ ]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_contrast (RandomContrast)     │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_brightness (RandomBrightness) │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg19 (Functional)                   │ (None, 7, 7, 512)           │      20,024,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 69,733,268 (266.01 MB)

 Trainable params: 23,244,422 (88.67 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 46,488,846 (177.34 MB)

### Evaluation on validation set

In [ ]:
file_paths = validation_dataset.file_paths
id_list = [int(filename.split("/")[-1].split(".")[0]) for filename in file_paths]
id_list

In [ ]:
predictions = model.predict(validation_dataset)
predicted_classes = np.argmax(predictions, axis=1)

val_df = pd.DataFrame({'id': id_list, 'predicted_label': predicted_classes})
metadata_df = pd.read_csv('/content/drive/MyDrive/COMP90086_2024_Project_train/train.csv')
merged_df = pd.merge(metadata_df, val_df, on='id', how='inner')
merged_df.to_csv('/content/drive/MyDrive/merged_df.csv', index=False)


import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, f1_score

overll_accuracy = accuracy_score(merged_df['stable_height'], merged_df['predicted_label'])
classification_report = classification_report(merged_df['stable_height'], merged_df['predicted_label'])

print(f"Overall accuracy: {overll_accuracy}")
print(f"Classification report:\n{classification_report}")

48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step
Overall accuracy: 0.529296875
Classification report:
              precision    recall  f1-score   support

           1       0.61      0.64      0.62       363
           2       0.62      0.59      0.60       382
           3       0.54      0.59      0.57       328
           4       0.43      0.39      0.41       228
           5       0.32      0.46      0.38       161
           6       0.00      0.00      0.00        74

    accuracy                           0.53      1536
   macro avg       0.42      0.44      0.43      1536
weighted avg       0.51      0.53      0.52      1536



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, f1_score

# Compute overall performance accuracy / classification_report
overll_accuracy = accuracy_score(merged_df['stable_height'], merged_df['predicted_label'])
classification_report = classification_report(merged_df['stable_height'], merged_df['predicted_label'])

print(f"Overall accuracy: {overll_accuracy}")
print(f"Classification report:\n{classification_report}")

# Compare the accuracy across different camera angle / Only cube versus different shapes / instability type / easy or hard
angle_accuracy = merged_df.groupby('cam_angle').apply(lambda x: accuracy_score(x['stable_height'], x['predicted_label'])).to_dict()
shape_accuracy = merged_df.groupby('shapeset').apply(lambda x: accuracy_score(x['stable_height'], x['predicted_label'])).to_dict()
instability_accuracy = merged_df.groupby('instability_type').apply(lambda x: accuracy_score(x['stable_height'], x['predicted_label'])).to_dict()
type_accuracy = merged_df.groupby('type').apply(lambda x: accuracy_score(x['stable_height'], x['predicted_label'])).to_dict()

# angle_f1 = merged_df.groupby('cam_angle').apply(lambda x: f1_score(x['stable_height'], x['predicted_label'], average="macro")).to_dict()
# shape_f1 = merged_df.groupby('shapeset').apply(lambda x: f1_score(x['stable_height'], x['predicted_label'], average="macro")).to_dict()
# instability_f1 = merged_df.groupby('instability_type').apply(lambda x: f1_score(x['stable_height'], x['predicted_label'], average="macro")).to_dict()
# type_f1 = merged_df.groupby('type').apply(lambda x: f1_score(x['stable_height'], x['predicted_label'], average="macro")).to_dict()

### Generate test label

In [ ]:
import tensorflow as tf

test_img_dir = '/content/drive/MyDrive/COMP90086_2024_Project_test/test'

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_img_dir,
    labels=None,
    shuffle=False,
    image_size=(DIM, DIM),
    batch_size=BATCH,
    label_mode=None
)


test_df['id'] = test_df['id'].astype(str)
test_df.sort_values("id", inplace=True)

predictions = model.predict(test_dataset)


predicted_classes = np.argmax(predictions, axis=1)


submission_df = pd.DataFrame({'id': test_df['id'], 'stable_height': predicted_classes})

submission_df['id'] = submission_df['id'].astype(int)
submission_df.sort_values("id", inplace=True)

submission_df.to_csv('/content/drive/MyDrive/COMP90086_2024_Project_test/test_predict_VGG.csv', index=False)


Found 1920 files.
60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 328ms/step
